In [ ]:
from __future__ import division

import math
import os
import sys
import tensorflow as tf
import tensorflow.contrib.slim as slim

import flowers_dataset
import inception_preprocessing
import inception_v1

In [ ]:
home_dir = os.path.expanduser('~')
base_data_dir = os.path.join(home_dir, "data/flowers")

dataset_dir = os.path.join(base_data_dir, "tfrecords")
dataset_split_name = "validation"

checkpoint_dir = os.path.join(base_data_dir, "train_logs/v006")

eval_loop = True
eval_dir = os.path.join(base_data_dir, "eval_logs/v006")

batch_size = 100
num_preprocessing_threads = 4

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
with tf.Graph().as_default():
    with tf.device('/cpu:0'):
        slim.create_global_step()

        # Create a dataset provider that loads data from the dataset
        dataset = flowers_dataset.get_dataset(dataset_split_name, dataset_dir)
        provider = slim.dataset_data_provider.DatasetDataProvider(dataset, num_epochs=1)
        [image, label] = provider.get(['image', 'label'])
        
        # Preprocess images.
        eval_image_size = inception_v1.inception_v1.default_image_size
        image = inception_preprocessing.preprocess_image(image, eval_image_size, eval_image_size, is_training=False)
        images, labels = tf.train.batch(
            [image, label],
            batch_size=batch_size,
            allow_smaller_final_batch=True,
            num_threads=num_preprocessing_threads)
        labels = tf.squeeze(labels)

        # Define the model.
        with slim.arg_scope(inception_v1.inception_v1_arg_scope(weight_decay=0.0)):
            logits, _ = inception_v1.inception_v1(images, num_classes=dataset.num_classes, is_training=False)
        predictions = tf.argmax(logits, 1)
        # Define the metrics.
        accuracy, update_op = slim.metrics.streaming_accuracy(predictions, labels)
        # Summary.
        summary_op = tf.scalar_summary("Accuracy", accuracy)

        if eval_loop:
            # Run evaluation loop.
            slim.evaluation.evaluation_loop(
                master="",
                checkpoint_dir=checkpoint_dir,
                logdir=eval_dir,
                num_evals=math.ceil(dataset.num_samples / float(batch_size)),
                eval_op=update_op,
                summary_op=summary_op,
                variables_to_restore=slim.get_variables_to_restore())
        else:
            checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
            metric_values = slim.evaluation.evaluate_once(
                master="",
                checkpoint_path=checkpoint_path,
                logdir=eval_dir,
                num_evals=math.ceil(dataset.num_samples / float(batch_size)),
                eval_op=update_op,
                final_op=accuracy,
                variables_to_restore=slim.get_variables_to_restore())
            print "Final accuracy: %f" % metric_values